In [103]:
pip install pandas geopy pyproj

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------- ----------------------- 2.6/6.3 MB 14.1 MB/s eta 0:00:01
   ------------------------------------ --- 5.8/6.3 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 14.0 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   -------------------- ------------------- 1/2 [pyproj]
   ---------------------------------------- 2/2 [pyproj]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from pyproj import Transformer
import numpy as np


In [158]:
import socket
socket.getaddrinfo('localhost', 8080) #resolve o problema de hostname [Errno 11002] ao executar as milhares de consultas de bairro pela geolocalização

[(<AddressFamily.AF_INET6: 23>, 0, 0, '', ('::1', 8080, 0, 0)),
 (<AddressFamily.AF_INET: 2>, 0, 0, '', ('127.0.0.1', 8080))]

Leitura do Dataset Condomínios

In [38]:
condominios = pd.read_csv("condominios.csv")
print(condominios.shape)
condominios.head(10)

(6373, 13)


,FID,id_condominio,nome_condominio,cartografia,endereco,idade,qtd_elevadores,padrao_acabamento_nome,tipo_uso,tipo_condominio,qtd_unidades,qtd_unidades_por_andar,the_geom
0,condominios.fid--5ab48429_18b14a3152e_-352b,3,Quarup,48-110-130,RUA TAVARES COUTINHO 1696,42,2.0,Médio,Residencial,Vertical,24.0,4.0,POINT (556847.4150239625 9587374.438595966)
1,condominios.fid--5ab48429_18b14a3152e_-352a,4,Mirafiori,48-110-172,RUA TAVARES COUTINHO 1736,24,2.0,Superior,Residencial,Vertical,30.0,2.0,POINT (556890.4950691864 9587363.927487098)
2,condominios.fid--5ab48429_18b14a3152e_-3529,5,Scopa Residenza,48-115-117,RUA FREI MANSUETO 333,9,2.0,Superior,Residencial,Vertical,69.0,3.0,POINT (556750.5011590382 9588024.013990007)
3,condominios.fid--5ab48429_18b14a3152e_-3528,6,Luciano Magalhães,48-26-354,RUA TAVARES COUTINHO 199,43,1.0,Médio,Residencial,Vertical,17.0,2.0,POINT (556930.6701409749 9587444.584830115)
4,condominios.fid--5ab48429_18b14a3152e_-3527,7,San Marino,48-26-371,RUA TAVARES COUTINHO 1747,39,2.0,Médio,Residencial,Vertical,26.0,2.0,POINT (556911.0396194621 9587443.661575532)
5,condominios.fid--5ab48429_18b14a3152e_-3526,8,Copenhague,48-26-393,RUA TAVARES COUTINHO 1739,42,0.0,Médio,Residencial,Vertical,19.0,3.0,POINT (556890.5406720614 9587442.98361419)
6,condominios.fid--5ab48429_18b14a3152e_-3525,9,Montreal I,48-26-448,RUA TAVARES COUTINHO 1699,44,2.0,Médio,Residencial,Vertical,18.0,4.0,POINT (556843.8704228845 9587441.197803127)
7,condominios.fid--5ab48429_18b14a3152e_-3524,10,Liberty Place,48-36-290,RUA TAVARES COUTINHO 2050,24,2.0,Médio,Residencial,Vertical,27.0,2.0,POINT (557242.8221800133 9587391.496451523)
8,condominios.fid--5ab48429_18b14a3152e_-3523,11,Tia Joana II,50-213-124,RUA SAO FRANCISCO DO LOTEAMENTO SAO JOSE 304,33,0.0,Proletário,Residencial,Vertical,4.0,2.0,POINT (544206.1463078993 9579888.937917681)
9,condominios.fid--5ab48429_18b14a3152e_-3522,12,Tia Joana III,50-239-123,RUA SAO FRANCISCO DO LOTEAMENTO SAO JOSE 670,33,0.0,Econômico,Residencial,Vertical,4.0,2.0,POINT (543807.3179881548 9580029.67325092)


Filtragem da Coluna Geopoint para lista: *[latitude, longitude]*

In [73]:
geopoint = condominios["the_geom"].str.replace("POINT", "").str.replace("(", "").str.replace(")", "").str.split()
geopoint

0       [556847.4150239625, 9587374.438595966]
1       [556890.4950691864, 9587363.927487098]
2       [556750.5011590382, 9588024.013990007]
3       [556930.6701409749, 9587444.584830115]
4       [556911.0396194621, 9587443.661575532]
                         ...                  
6368     [552239.9697916809, 9585348.13258414]
6369     [547642.3796855016, 9583422.71139127]
6370    [547688.0860597235, 9583391.824024865]
6371    [557419.6769286944, 9582392.766334554]
6372      [557753.9092624151, 9586859.3180984]
Name: the_geom, Length: 6373, dtype: object

Convertendo do sistema UTM para Latitude Longitude

In [120]:
transformer = Transformer.from_crs("EPSG:32724", "EPSG:4326", always_xy=True)

In [188]:
tuplaGeopoint = []

for point in geopoint:
    tuplaGeopoint.append( transformer.transform( float(point[0]), float(point[1]) ))

tuplaGeopoint[:10]

geopoints = [(lat, lon) for (lon, lat) in tuplaGeopoint] #é necessario inverter as colunas pois o sistema disposto é invertido
                                                         #já o sistema atual EPSG é Latidude, Longitude
geopoints[:10]

[(-3.732950297999232, -38.48805449308904),
 (-3.7330451606149113, -38.48766648645134),
 (-3.7270743803449737, -38.488930632436414),
 (-3.732315277120804, -38.48730511839091),
 (-3.732323732830626, -38.48748189341989),
 (-3.7323299740172033, -38.48766649022929),
 (-3.732346375135369, -38.48808676320801),
 (-3.732793894890859, -38.4844937982232),
 (-3.800728522318616, -38.60186390612826),
 (-3.799456986611557, -38.6054564149804)]

In [184]:
geopoints_to_export = pd.DataFrame(np.array(geopoints), columns=["latitude", "longitude"])
geopoints_to_export.to_csv("geopoints.csv", index=False)

Busca de Bairro por geolocalização

In [175]:
somePoints = geopoints[:100]
somePoints

[(-3.732950297999232, -38.48805449308904),
 (-3.7330451606149113, -38.48766648645134),
 (-3.7270743803449737, -38.488930632436414),
 (-3.732315277120804, -38.48730511839091),
 (-3.732323732830626, -38.48748189341989),
 (-3.7323299740172033, -38.48766649022929),
 (-3.732346375135369, -38.48808676320801),
 (-3.732793894890859, -38.4844937982232),
 (-3.800728522318616, -38.60186390612826),
 (-3.799456986611557, -38.6054564149804),
 (-3.8013281695894126, -38.60075309724794),
 (-3.733141802469878, -38.48208888561246),
 (-3.7416674724116725, -38.48012351140471),
 (-3.7350228043200335, -38.48290662105191),
 (-3.7446264444536985, -38.47065123181685),
 (-3.7445760917458317, -38.47105953987306),
 (-3.7448179418522476, -38.47016104849294),
 (-3.727850657481922, -38.463680338070475),
 (-3.728166504147733, -38.46369004892053),
 (-3.732224001099402, -38.46171850513163),
 (-3.7492517912132297, -38.45405432635804),
 (-3.7459321576037286, -38.48311666370157),
 (-3.746203988357183, -38.488559746309875),

In [192]:
geolocator = Nominatim(user_agent="geolocate_bairro")

bairros = []
subconjunto_bairros = []
counter = 0

for point in geopoints:
# for point in somePoints:

    # print(point)
    
    location = geolocator.reverse(f"{point[0]}, {point[1]}", exactly_one=True, timeout=10)                                                                                                          #pois o outro sistema UTM é longitude, latitude

    if location:
        address = location.raw.get('address', {})
        bairro = address.get("neighbourhood") or address.get("suburb") or address.get("district")
        subconjunto_bairros.append(bairro)
    else:
        print("Localização não encontrada")

    counter = counter + 1
    if counter%100 == 0:
        bairros.append(subconjunto_bairros)
        print(subconjunto_bairros)
        subconjunto_bairros = []

['Varjota', 'Varjota', 'Mucuripe', 'Varjota', 'Varjota', 'Varjota', 'Varjota', 'Varjota', 'Bom Jardim', 'Siqueira', 'Bom Jardim', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Papicu', 'Vicente Pinzón', 'Vicente Pinzón', 'Vicente Pinzón', 'Praia do Futuro II', 'Cocó', 'Cocó', 'Engenheiro Luciano Cavalcante', 'Engenheiro Luciano Cavalcante', 'Conjunto Santa Luzia do Cocó', 'Engenheiro Luciano Cavalcante', 'Guararapes', 'Guararapes', 'Guararapes', 'São João do Tauape', 'Guararapes', 'Conjunto Santa Luzia do Cocó', 'Edson Queiroz', 'Edson Queiroz', 'Edson Queiroz', 'Sapiranga / Coité', 'Cambeba', 'Cambeba', 'Boa Vista/Castelão', 'Messejana', 'Messejana', 'Jardim Violeta', 'Cocó', 'De Lourdes', 'Praia de Iracema', 'Carlito Pamplona', 'Cocó', 'Meireles', 'Centro', 'Meireles', 'Aldeota', 'Meireles', 'Dias Macêdo', 'Carlito Pamplona', 'Mucuripe', 'Meireles', 'Carlito Pamplona', 'Centro', 'Praia do Futuro I', 'Meireles', 'Bela Vista', 'Engenheiro Luciano Cavalcante', 'São João do Tauape',

In [205]:
bairros.append(subconjunto_bairros)

In [206]:
len(bairros)

64

In [207]:
bairros_new = np.array(bairros)
bairros_new.shape
bairros_new = bairros_new.flatten()
bairros_new

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.

In [214]:
bairros_final = np.append(bairros_new, np.array(subconjunto_bairros))

In [216]:
subconjunto_bairros

['Rodolfo Teófilo',
 'São João do Tauape',
 'Presidente Kennedy',
 'Sapiranga / Coité',
 'Novo Mondubim',
 'Granja Portugal',
 'Granja Lisboa',
 'Granja Lisboa',
 'Genibaú',
 'Vicente Pinzón',
 'Vicente Pinzón',
 'Floresta',
 'Floresta',
 'Conjunto Ceará I',
 'Conjunto Esperança',
 'Conjunto Esperança',
 'Conjunto Cristo Redentor',
 'Conjunto Cristo Redentor',
 'Farias Brito',
 'Farias Brito',
 'Farias Brito',
 'Farias Brito',
 'Farias Brito',
 'Farias Brito',
 'Farias Brito',
 'Floresta',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Mondubim',
 'Cambeba',
 'Conjunto Campo dos Ingleses',
 'São Gerardo',
 'Presidente Kennedy',
 'Montese',
 'Messejana',
 'Montese',
 'Montese',
 'Montese',
 'Prefeito José Walter',
 'Prefeito José Walter',
 'Boa Vista/Castelão',
 'Granja Lisboa',
 'Serrinha',
 'Montese',
 'Bela Vista',
 'Demócrito Rocha',
 'Parangaba',
 'Bonsucesso',
 'Dom Lustosa',
 'Carlito Pamplona',
 'Monte Castelo',
 'Joaquim Távora',
 '

In [215]:
bairros_final

array(['Varjota', 'Varjota', 'Mucuripe', ..., 'Jóquei Clube',
       'Engenheiro Luciano Cavalcante', 'Papicu'],
      shape=(6373,), dtype='<U35')

In [217]:
bairros_to_export = pd.DataFrame(bairros_final, columns=["bairros"])
bairros_to_export.to_csv("bairros.csv", index=False)

In [193]:
bairros_to_export = pd.DataFrame(np.array(bairros), columns=["bairros"])
bairros_to_export.to_csv("bairros.csv", index=False)

ValueError: Shape of passed values is (63, 100), indices imply (63, 1)

Join do dataset original e mapeamento dos bairros respectivos

In [2]:
condominios = pd.read_csv("condominios.csv")
bairros = pd.read_csv("bairros.csv")

In [3]:
condominios

,FID,id_condominio,nome_condominio,cartografia,endereco,idade,qtd_elevadores,padrao_acabamento_nome,tipo_uso,tipo_condominio,qtd_unidades,qtd_unidades_por_andar,the_geom
0,condominios.fid--5ab48429_18b14a3152e_-352b,3,Quarup,48-110-130,RUA TAVARES COUTINHO 1696,42,2.0,Médio,Residencial,Vertical,24.0,4.0,POINT (556847.4150239625 9587374.438595966)
1,condominios.fid--5ab48429_18b14a3152e_-352a,4,Mirafiori,48-110-172,RUA TAVARES COUTINHO 1736,24,2.0,Superior,Residencial,Vertical,30.0,2.0,POINT (556890.4950691864 9587363.927487098)
2,condominios.fid--5ab48429_18b14a3152e_-3529,5,Scopa Residenza,48-115-117,RUA FREI MANSUETO 333,9,2.0,Superior,Residencial,Vertical,69.0,3.0,POINT (556750.5011590382 9588024.013990007)
3,condominios.fid--5ab48429_18b14a3152e_-3528,6,Luciano Magalhães,48-26-354,RUA TAVARES COUTINHO 199,43,1.0,Médio,Residencial,Vertical,17.0,2.0,POINT (556930.6701409749 9587444.584830115)
4,condominios.fid--5ab48429_18b14a3152e_-3527,7,San Marino,48-26-371,RUA TAVARES COUTINHO 1747,39,2.0,Médio,Residencial,Vertical,26.0,2.0,POINT (556911.0396194621 9587443.661575532)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6368,condominios.fid--5ab48429_18b14a3152e_-1c4b,243700,L'acqua di Fátima Condominium,14-162-260,RUA PADRE LEOPOLDO FERNANDES (FATIMA) 178,7,3.0,Superior,Residencial,Vertical,64.0,4.0,POINT (552239.9697916809 9585348.13258414)
6369,condominios.fid--5ab48429_18b14a3152e_-1c4a,243701,Duo Towers Residence,36-171-347,AVENIDA SENADOR FERNANDES TAVORA 6770,6,2.0,Médio,Residencial,Vertical,60.0,6.0,POINT (547642.3796855016 9583422.71139127)
6370,condominios.fid--5ab48429_18b14a3152e_-1c49,243702,Duo Towers Office,36-171-145,AVENIDA AMERICO BARREIRA 6770 A,6,8.0,Médio,Comercial,Vertical,190.0,19.0,POINT (547688.0860597235 9583391.824024865)
6371,condominios.fid--5ab48429_18b14a3152e_-1c48,243750,José Alencar Ramos 385,56-184-29,RUA JOSE ALENCAR RAMOS 385,9,0.0,Médio,Comercial,Vertical,17.0,4.0,POINT (557419.6769286944 9582392.766334554)


In [4]:
bairros

,bairros
0,Varjota
1,Varjota
2,Mucuripe
3,Varjota
4,Varjota
...,...
6368,Fátima
6369,Jóquei Clube
6370,Jóquei Clube
6371,Engenheiro Luciano Cavalcante


In [5]:
condominios["bairros"] = bairros

In [15]:
condominios

,FID,id_condominio,nome_condominio,cartografia,endereco,idade,qtd_elevadores,padrao_acabamento_nome,tipo_uso,tipo_condominio,qtd_unidades,qtd_unidades_por_andar,the_geom,bairros
0,condominios.fid--5ab48429_18b14a3152e_-352b,3,Quarup,48-110-130,RUA TAVARES COUTINHO 1696,42,2.0,Médio,Residencial,Vertical,24.0,4.0,POINT (556847.4150239625 9587374.438595966),Varjota
1,condominios.fid--5ab48429_18b14a3152e_-352a,4,Mirafiori,48-110-172,RUA TAVARES COUTINHO 1736,24,2.0,Superior,Residencial,Vertical,30.0,2.0,POINT (556890.4950691864 9587363.927487098),Varjota
2,condominios.fid--5ab48429_18b14a3152e_-3529,5,Scopa Residenza,48-115-117,RUA FREI MANSUETO 333,9,2.0,Superior,Residencial,Vertical,69.0,3.0,POINT (556750.5011590382 9588024.013990007),Mucuripe
3,condominios.fid--5ab48429_18b14a3152e_-3528,6,Luciano Magalhães,48-26-354,RUA TAVARES COUTINHO 199,43,1.0,Médio,Residencial,Vertical,17.0,2.0,POINT (556930.6701409749 9587444.584830115),Varjota
4,condominios.fid--5ab48429_18b14a3152e_-3527,7,San Marino,48-26-371,RUA TAVARES COUTINHO 1747,39,2.0,Médio,Residencial,Vertical,26.0,2.0,POINT (556911.0396194621 9587443.661575532),Varjota
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6368,condominios.fid--5ab48429_18b14a3152e_-1c4b,243700,L'acqua di Fátima Condominium,14-162-260,RUA PADRE LEOPOLDO FERNANDES (FATIMA) 178,7,3.0,Superior,Residencial,Vertical,64.0,4.0,POINT (552239.9697916809 9585348.13258414),Fátima
6369,condominios.fid--5ab48429_18b14a3152e_-1c4a,243701,Duo Towers Residence,36-171-347,AVENIDA SENADOR FERNANDES TAVORA 6770,6,2.0,Médio,Residencial,Vertical,60.0,6.0,POINT (547642.3796855016 9583422.71139127),Jóquei Clube
6370,condominios.fid--5ab48429_18b14a3152e_-1c49,243702,Duo Towers Office,36-171-145,AVENIDA AMERICO BARREIRA 6770 A,6,8.0,Médio,Comercial,Vertical,190.0,19.0,POINT (547688.0860597235 9583391.824024865),Jóquei Clube
6371,condominios.fid--5ab48429_18b14a3152e_-1c48,243750,José Alencar Ramos 385,56-184-29,RUA JOSE ALENCAR RAMOS 385,9,0.0,Médio,Comercial,Vertical,17.0,4.0,POINT (557419.6769286944 9582392.766334554),Engenheiro Luciano Cavalcante


In [16]:
condominios.to_csv("condominios_mapeados.csv", index=False)

In [ ]:
latlong = pd.read_csv("geopoints.csv")